# Graphs and Plots
* Displaying some graphical information to the user.
* Note: I seperated the histogram plot in a different notebook because the histogram is slowing down the execution of all the other graphs and interface.

## xy graph
This graph shows the robot’s position.

In [1]:
import ipywidgets as widgets
from geometry_msgs.msg import Twist
from matplotlib import animation, rc
from nav_msgs.msg import Odometry
import rospy
import jupyros as jr
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
from rt2_assignment1.srv import Command

%matplotlib widget
# Initializing 'plotting' node
rospy.init_node('plotting')

# First set up the figure and the axis 
fig, ax = plt.subplots()
ax.set_xlim(( -5, 5))
ax.set_ylim((-5, 5))
line, = ax.plot([], [], 'ro')

# Defining position variables
x_data=[]
y_data=[]

# Initializing line
def init():
    line.set_data([], [])
    return (line,)

## These following parts are merged in the second cell (Robot velocities) ##

#def odom_callback(msg):
#    y_data.append(msg.pose.pose.position.y)
#   x_data.append(msg.pose.pose.position.x)

#jr.subscribe('/odom', Odometry, odom_callback)

def animate(i):
    line.set_data(x_data, y_data)
    return (line,)

# call the animator 
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=20, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Robot Velocities
This plot is for visualizing cmd_vel vs. actual velocity (for linear and angular velocity).

In [2]:
# Setting up the figure and the axis 
fig_vel, ax_vel = plt.subplots()

Tmin = 0 # The min value of x_axis
Tmax = 10 # The max value of x_axis
ax_vel.set_xlim(Tmin, Tmax)
ax_vel.set_ylim((-1.1, 1.1))
plt.title("Velocity")
plt.xlabel("Time")
plt.ylabel("Velocity")

# Define some variables
t0 = rospy.get_time() # The moment of start plotting
cmd_x = 0 # cmd_vel linear.x
cmd_z = 0 # cmd_vel angualr.z


# Define lines for plotting
line_vel_cmd, = ax_vel.plot([], [], label = "line_cmd") 
line_vel_actual, = ax_vel.plot([], [], label = "line_act") 
line_velA_cmd, = ax_vel.plot([], [], label = "ang_cmd") 
line_velA_actual, = ax_vel.plot([], [], label = "ang_act") 

# Initializing line Data
t_data= []
v_linear_data=[]
v_linearCMD_data=[]
v_angular_data=[]
v_angularCMD_data=[]

def init2():
    global t0
    t0 = rospy.get_time()
    line_vel_actual.set_data([], [])
    line_vel_cmd.set_data([], [])
    line_velA_actual.set_data([], [])
    line_velA_cmd.set_data([], [])

    return (line_vel_actual, line_vel_cmd, line_velA_actual, line_velA_cmd,)

## odom call_back  ##
def odom_callback(msg):
    global t0, Tmax, Tmin, cmd_x, cmd_z
    temp = rospy.get_time()
    
    # Updating odometry velocities
    v_linear_data.append(msg.twist.twist.linear.x)
    v_angular_data.append(-1*msg.twist.twist.angular.z)
    
    # Adapting y_axis
    tt = temp-t0
    t_data.append(tt)
    if tt > Tmax:
        Tmin = Tmin + 1
        Tmax = Tmax + 1
        ax_vel.set_xlim(Tmin, Tmax)
        
    # Updating position data for xy graph
    y_data.append(msg.pose.pose.position.y)
    x_data.append(msg.pose.pose.position.x)
    # Updating velocity data for velocities graph
    v_linearCMD_data.append(cmd_x)
    v_angularCMD_data.append(cmd_z)    

## Tiwist call_back ##   
def twist_callback(msg_twist):
    global cmd_x, cmd_z
    # Updating cmd_vel values
    cmd_x = msg_twist.linear.x
    cmd_z = msg_twist.angular.z
       
def animate2(i):
    # add legend
    ax_vel.legend([line_vel_actual, line_vel_cmd, line_velA_actual, line_velA_cmd], 
                  ['Linear Vel.', 'Cmd. linear','Angular Vel.','Cmd. angular'], loc = 'lower right')
    # Update lines
    line_vel_actual.set_data(t_data, v_linear_data)
    line_vel_cmd.set_data(t_data, v_linearCMD_data)
    line_velA_actual.set_data(t_data, v_angular_data)
    line_velA_cmd.set_data(t_data, v_angularCMD_data)
    return (line_vel_actual, line_vel_cmd, line_velA_actual, line_velA_cmd,)#

# Subscribing for /odom and /cmd_vel topics
jr.subscribe('/odom', Odometry, odom_callback)
jr.subscribe('/cmd_vel', Twist, twist_callback)

# call the animator. blit=True means only re-draw the parts that have changed.
anim2 = animation.FuncAnimation(fig_vel, animate2, init_func=init2,
                                frames=1000,blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Reached/Cancelled targets 
Plotting a bar plot for displaying the number of reached targets and cancelled targets.

In [3]:
import rt2_assignment1.msg
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import rospy
import jupyros as jr
import numpy as np


# Defining barPlot Data
goals = [0, 0] # goals[0] for reached and goals[1] for cancelled
labels = ['Reached', 'Cancelled']

# Defining plot figure and its parameters
fig_b, ax_b = plt.subplots()
ax_b.yaxis.set_ticks(np.arange(0, max(goals)+2, 1))
plt.ylabel('number of goals')
plt.title('Plot for Goal Results')
#plt.show()
bars = plt.bar(labels, goals)

# Callback for PositionActionResult msg
def result_callback(msg):
    # Update bar and histogram data
    if msg.result.ok == True:
        goals[0] = goals[0] + 1 
        HIST_data.append(msg.header.stamp.secs - msg.status.goal_id.stamp.secs)
    else:
        goals[1] = goals[1] + 1 
        

def animate_rects(i):
    # Updating the bar figure
    for b, val in zip(bars, goals):
         b.set_height(val)   
    ax_b.yaxis.set_ticks(np.arange(0, max(goals)+2, 1))
    return bars, 

# Subscribe to the /go_to_point/results topic
jr.subscribe('/go_to_point/result', rt2_assignment1.msg.PositionActionResult, result_callback)

# Create an animation function for the barPlot
anim_goal = animation.FuncAnimation(fig_b, animate_rects, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …